In [8]:
import os 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.io as sio
import shdom
from scipy.ndimage import center_of_mass


In [9]:
out_dir = '../experiments/rico32x37x26_velocity_0_0_1img_perspective/monochromatic'
dynamic_medium, rte_solver, measurements = shdom.load_dynamic_forward_model('experiments/rico32x37x26_velocity_0_0_1img_perspective/dynamic_medium')

In [10]:
static_rte_solver = rte_solver.solver_list[0]
first = True
for medium in dynamic_medium.dynamic_medium:
    if first:
        lwc = medium.scatterers['cloud'].lwc.data
        reff = medium.scatterers['cloud'].reff.data
        veff = medium.scatterers['cloud'].veff.data
        first = False
    else:
        lwc += medium.scatterers['cloud'].lwc.data
        reff += medium.scatterers['cloud'].reff.data
        veff += medium.scatterers['cloud'].veff.data
lwc /= len(dynamic_medium.dynamic_medium)
reff /= len(dynamic_medium.dynamic_medium)
veff /= len(dynamic_medium.dynamic_medium)
lwc = shdom.GridData(medium.scatterers['cloud'].lwc.grid,lwc)
reff = shdom.GridData(medium.scatterers['cloud'].reff.grid,reff)
reff.data[reff.data<1] =0
veff = shdom.GridData(medium.scatterers['cloud'].veff.grid,veff)
if np.isscalar(veff.data):
    # veff.data = veff.data
    print()
else:
    veff.data[veff.data<0.01]=0

In [11]:
mie = shdom.MiePolydisperse()
mie.read_table(file_path='../mie_tables/polydisperse/Water_672nm.scat')#set the correct mie
avg_scatterer = shdom.MicrophysicalScatterer(lwc,reff,veff)
avg_scatterer.add_mie(mie)
atmospheric_grid = avg_scatterer.grid + medium.scatterers['air'].grid
atmosphere = shdom.Medium(atmospheric_grid)
atmosphere.add_scatterer(avg_scatterer, name='cloud')
atmosphere.add_scatterer(medium.scatterers['air'], name='air')

Reading mie table from file: ../mie_tables/polydisperse/Water_672nm.scat


In [12]:
static_measurement = shdom.Measurements(measurements.camera,measurements.images,measurements.pixels,measurements.wavelength)

shdom.save_forward_model(out_dir, atmosphere, static_rte_solver, static_measurement)
